# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

A: This is a classification problem because we are trying to classify a label; if a student is going to pass or fail a subject. A regression problem might try to predict the grade a student would achieve.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] -1
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = len(student_data[student_data['passed'] == 'no'])
grad_rate = 1.0 * n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate * 100)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']



### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
from sklearn.cross_validation import StratifiedShuffleSplit, train_test_split

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=1)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])

# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Code for outputting training / testing results

from IPython.display import HTML

def scores_table_header():
    output = "<table><thead><tr>"
    output += "<th>size</th><th>train</th><th>test</th>"
    output += "<th>train time</th><th>test time</th>"
    output += "</th></thead><tbody>"
    
    return output

def scores_table_footer():
    return "</tbody></table>"

def class_results_row(result, klass):
    #output = "<tr><td><strong> {} </strong></td>".format(klass)
    output = ""
    for size in [100, 200, 300]:
        output += "<tr>"
        output += "<td> {} </td>".format(size)
        output += "<td> {:.4f} </td>".format(result[size]['f1_train'])
        output += "<td> {:.4f} </td>".format(result[size]['f1_test'])
        output += "<td> {:.3f} </td>".format(result[size]['train_time'])
        output += "<td> {:.3f} </td>".format(result[size]['predict_time'])

        #output += "<td> {:.4f} </td>".format(result[size]['f1_train'])
        #output += "<td> {:.4f} </td>".format(result[size]['f1_test'])
        #output += "<td> {:.3f} </td>".format(result[size]['train_time'])
        #output += "<td> {:.3f} </td>".format(result[size]['predict_time'])
        output += "</tr>"
    return output

results = {}

### Decision Trees

* General Applications:
 * Easy to understand classification. Best suited when some features play big roles in separating large parts of the data
 * Can determine weather a person will like a restaurant, based on some inputs like "type of food", "atmosphere", "price", "popularity"
 * Used to create the game "20 Questions" where the game automaticall chooses what object a person is thinking of by asking up to 20 questions.
* strenghts / weaknesses
 * prone to overfitting with data that contains too many features creating a complicated decision tree
   * Response from previous review feedback: [reference to the above point](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345404/modules/544698886575460/lessons/5450810003/concepts/24497085540923)
 * Very fast at prediction time
 * Simple, can be printed out and interpreted by a human
* Why? Maybe some features are important in classifying the students grad rate that will create large splits in the trees.


In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return end - start

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion="entropy")

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.004


0.003576993942260742

In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return [f1_score(target.values, y_pred, pos_label='yes'), end - start]

train_f1_score = predict_labels(clf, X_train, y_train)[0]
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [10]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[0])

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.703125


In [11]:
# Train and predict using different training set sizes
klass = clf.__class__.__name__
results[klass] = {}
    
def train_predict(clf, X_train, y_train, X_test, y_test):
    local_results = {
        'size': len(X_train)
    }
    
    print "------------------------------------------"
    print "Training set size: {}".format(local_results['size'])
    local_results['train_time'] = train_classifier(clf, X_train, y_train)

    local_results['f1_train'], _ = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {}".format(local_results['f1_train'])

    local_results['f1_test'], local_results['predict_time'] = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(local_results['f1_test'])
    return local_results

for num in [100, 200, 300]:
    results[klass][num] = train_predict(clf, X_train[:num], y_train[:num], X_test, y_test)
    

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.677966101695
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.782608695652
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 

In [12]:
print "class:", klass
output = scores_table_header()
output += class_results_row(results[klass], klass)
output += scores_table_footer()
HTML(output)

class: DecisionTreeClassifier


size,train,test,train time,test time
100,1.0000,0.6780,0.002,0.000
200,1.0000,0.7826,0.002,0.000
300,1.0000,0.7077,0.003,0.000


### SVM
* General Applications
  * Great for classification where a good margin exists between classes of data.
  * Typical uses include
    * text classification
    * image classification
    * handwritten character recognition
* [strengths / weaknesses](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345404/modules/544698886575460/lessons/5447009165/concepts/23841887100923)
 * work really well in complicated domains with clear margin of separation
 * not great with too much noise
 * not great with large data sets because training time would be too long
* Why? We've got a small training set and `SVM` seems like a good model to try out.


In [13]:
# Train a model
import time

from sklearn import svm
clf = svm.SVC(kernel="rbf")

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.010


0.010365962982177734

In [14]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return [f1_score(target.values, y_pred, pos_label='yes'), end - start]

train_f1_score, _ = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for training set: 0.858387799564


In [15]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[0])

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.846153846154


In [16]:
# Train and predict using different training set sizes
klass = clf.__class__.__name__
results[klass] = {}
    
for num in [100, 200, 300]:
    results[klass][num] = train_predict(clf, X_train[:num], y_train[:num], X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.859060402685
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.833333333333
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.858064516129
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.84076433121
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.009
Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
F1 score for training set: 0.858387799564
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.846153846154


In [17]:
print "class:", klass
output = scores_table_header()
output += class_results_row(results[klass], klass)
output += scores_table_footer()
HTML(output)

class: SVC


size,train,test,train time,test time
100,0.8591,0.8333,0.002,0.001
200,0.8581,0.8408,0.005,0.002
300,0.8584,0.8462,0.009,0.003


### K-NN

* General Applications:
  * Great for classification of data with complex decision boundaries, when there is not too much training data.
  * Very popular in recommender systems
  * Used in Concept Search - searching for documents containing similar topics
* strengths / weaknesses
  * good when training data is large
  * need to figure out parameter K
  * predictions are memory and computationally expensive with lots of training data
* Why? We have a small enough training set to use in the prediciton step. 


In [18]:
# Train a model
import time

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training KNeighborsClassifier...
Done!
Training time (secs): 0.002


0.0016100406646728516

In [19]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.010
F1 score for training set: [0.85581395348837208, 0.01007699966430664]


In [20]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: [0.76811594202898548, 0.0035991668701171875]


In [21]:
# Train and predict using different training set sizes
klass = clf.__class__.__name__
results[klass] = {}

for num in [100, 200, 300]:
    results[klass][num] = train_predict(clf, X_train[:num], y_train[:num], X_test, y_test)

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.788321167883
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.772727272727
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.834482758621
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.797101449275
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.008
F1 score for training

In [22]:
print "class:", klass
output = scores_table_header()
output += class_results_row(results[klass], klass)
output += scores_table_footer()
HTML(output)

class: KNeighborsClassifier


size,train,test,train time,test time
100,0.7883,0.7727,0.001,0.002
200,0.8345,0.7971,0.001,0.002
300,0.8558,0.7681,0.001,0.003


### F<sub>1</sub> Scores for different models / train sizes table


In [23]:
output = scores_table_header()

for klass in results:
    output += "<tr><td colspan='5'>" + klass + "</td></tr>"
    output += class_results_row(results[klass], klass)
    
output += scores_table_footer()

HTML(output)


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

The Support Vector Classifier was the best performing model with an F<sub>1</sub> score of 0.8462 when testing on 300 samples. Compared to a score of 0.7681 for K Nearest Neighbors and 0.7077 for Decision Trees.

The resources, cost and performanc were not a major factor for the given data. This is due to the small amount of data in this example. Although all training and prediction times were very low, training was shortest for K Nearest Neighbors at 0.001 seconds for 300 samples because it does it's computation at prediction time, then decision trees at 0.003 seconds for 300 samples, and the SVC at 0.009 seconds for 300 samples was the slowest. Prediction time was shortest for decision trees, then the SVC and slowest for k-nearest neighbors.


Support Vector classification works by linearly separating classes of data. A linear separator is chosen by finding the largest margin between classes. Data points that are closest to the margins on both sides of the separator are considerted the support vecotors.

When a linear separation of data cannot be accomplished, SVM uses a kernel trick function to map the input data points into a new set of data that can be separated. See the illustration below how a non linearly seperable set of data is transformed.

![SVM Illustration](SVM-illustration.jpg)

Illustration credit: http://users.sussex.ac.uk/~christ/crs/ml/lec08a.html

In [24]:
# GridSearch tuning

from sklearn import svm, grid_search, datasets

parameters = {'kernel': ('linear', 'rbf'), 'C': [1.0, 10.0], 'gamma': ('auto', 0.01, 0.1, 0.5)}

svr = svm.SVC()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
clf
clf.best_params_


{'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}

In [25]:
from sklearn.metrics import f1_score
y_pred = clf.predict(X_test)
f1 = f1_score(y_test.values, y_pred, pos_label='yes')
f1

0.84810126582278489

### `GridSearch` results

Using `GridSearch`, a slightly better F<sub>1</sub> score of `0.8481` was achived on the test data.

Interestingly my initial parameters to `GridSearch` provided a model that performed worse than the `SVM` model I used earlier. I'm guessing this is because `GridSearch` is using `cross-validation` which is fitting on a subset of the training data,  providing worse than ideal results. The worse performing model is shown below.

In [42]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Create the parameters list you wish to tune
svc = svm.SVC()
parameters = {'kernel': ('linear', 'rbf'), 'C': [1.0, 10.0], 'gamma': ('auto', 0.01, 0.1, 0.5)}

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(svc, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[0])
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[0])

Predicting labels using SVC...
Done!
Prediction time (secs): 0.010
Tuned model has a training F1 score of 0.9754.
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Tuned model has a testing F1 score of 0.8481.


In [44]:
# Worse performing GridSearch tuning
parameters = {'kernel': ('linear', 'rbf'), 'C': [1.0, 1.5, 10.0], 'gamma': ('auto', 0.01, 0.1, 0.5)}

svr = svm.SVC()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
clf
clf.best_params_


{'C': 1.5, 'gamma': 'auto', 'kernel': 'rbf'}

In [27]:
from sklearn.metrics import f1_score
y_pred = clf.predict(X_test)
f1 = f1_score(y_test.values, y_pred, pos_label='yes')
f1

0.84563758389261745

In [30]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, pos_label="yes")
gs = grid_search.GridSearchCV(svc, tuned_parameters, scoring=f1_scorer)
gs.fit(X_train, y_train)

NameError: name 'svc' is not defined